![logo](https://raw.githubusercontent.com/goodmami/wn/main/docs/_static/wn-logo-rotate.svg)

# Wn Demonstration

This is a demonstration of the [Wn](https://github.com/goodmami/wn/) library for working with wordnets in Python. To run this notebook locally, you will need to use [Python 3.6](https://www.python.org/) or higher and install the `wn` and `jupyter` packages, and download some wordnet data:

* Linux/macOS

  ```console
  $ python3 -m pip install wn jupyter
  $ python3 -m wn download omw oewn:2021
  ```
  
* Windows

  ```console
  > py -3 -m pip install wn jupyter
  > py -3 -m wn download omw oewn:2021
  ```

Now you should be able to import the `wn` package:

In [1]:
import wn

## Primary Queries

A **primary query** of the database is when basic parameters such as word forms, parts of speech, or public identifiers (e.g., synset IDs) are used to retrieve basic wordnet entities. You can perform these searches via module-level functions such as [wn.words()](https://wn.readthedocs.io/en/latest/api/wn.html#wn.words), [wn.senses()](https://wn.readthedocs.io/en/latest/api/wn.html#wn.senses), and [wn.synsets()](https://wn.readthedocs.io/en/latest/api/wn.html#wn.synsets):

In [2]:
wn.words('Malacca')

[Word('oewn-Malacca-n')]

In [3]:
wn.synsets('Malacca')

[Synset('oewn-08985168-n')]

### Filtering by Language / Lexicon

Once you've added multiple wordnets, however, you will often get many results for such queries. If that's not clear, then the following will give you some idea(s):

In [4]:
wn.words('idea')

[Word('omw-en-idea-n'),
 Word('omw-sk-idea-n'),
 Word('omw-pl-idea-n'),
 Word('omw-is-ídea-n'),
 Word('omw-zsm-idea-n'),
 Word('omw-iwn-idea-n'),
 Word('omw-it-idea-n'),
 Word('omw-gl-idea-n'),
 Word('omw-fi-idea-n'),
 Word('omw-ca-idea-n'),
 Word('omw-eu-idea-n'),
 Word('omw-es-idea-n'),
 Word('oewn-idea-n')]

You can filter down the results by language, but that may not be enough if you have multiple wordnets for the same language (e.g., the [OMW English Wordnet based on WordNet 3.0](https://github.com/omwn/omw-data/) and the [Open English WordNet](https://en-word.net/)):

In [5]:
wn.words('idea', lang='en')

[Word('omw-en-idea-n'), Word('oewn-idea-n')]

The [wn.lexicons()](https://wn.readthedocs.io/en/latest/api/wn.html#wn.lexicons) function can show which lexicons have been added for a language:

In [6]:
wn.lexicons(lang='en')

[<Lexicon omw-en:1.4 [en]>, <Lexicon oewn:2021 [en]>]

You can use the `id:version` string to restrict queries to a particular lexicon:

In [7]:
wn.words('idea', lexicon='omw-en:1.4')

[Word('omw-en-idea-n')]

But it can become tedious to enter these specifiers each time. Instead, a [wn.Wordnet](https://wn.readthedocs.io/en/latest/api/wn.html#the-wordnet-class) object can be used to make the language/lexicon filters persistent:

In [8]:
en = wn.Wordnet(lexicon='omw-en:1.4')
en.words('idea')

[Word('omw-en-idea-n')]

### Filtering by Word Form and Part of Speech

Even within a single lexicon a word may return multiple results:

In [9]:
en.words('pencil')

[Word('omw-en-pencil-n'), Word('omw-en-pencil-v')]

You can restrict results by part of speech, as well. E.g., to get the verbal sense of *pencil* (e.g., *to pencil in an appointment*), use the `pos` filter:

In [10]:
en.words('pencil', pos='v')

[Word('omw-en-pencil-v')]

This works for getting senses and synsets, too:

In [11]:
en.senses('pencil')

[Sense('omw-en-pencil-03908204-n'),
 Sense('omw-en-pencil-14796748-n'),
 Sense('omw-en-pencil-13863020-n'),
 Sense('omw-en-pencil-03908456-n'),
 Sense('omw-en-pencil-01688604-v')]

In [12]:
en.senses('pencil', pos='v')

[Sense('omw-en-pencil-01688604-v')]

In [13]:
en.synsets('pencil', pos='v')

[Synset('omw-en-01688604-v')]

The wordform itself is just a filter on the results. Leaving it off, you can get all results for a particular part of speech:

In [14]:
len(en.words(pos='v'))

11531

Or all results, regardless of the part of speech:

In [15]:
len(en.words())

156584

## Secondary Queries

**Secondary queries** are used when you want to get additional information from a retrieved entity, such as the forms of a word or the definition of a synset. They are also used for finding links between entities, such as the senses of a word or the relations of a sense or synset.

In [16]:
pencil = en.words('pencil', pos='v')[0]
pencil.lemma()

'pencil'

In [17]:
pencil.forms()

['pencil', 'pencilled', 'pencilling']

In [18]:
pencil.pos

'v'

In [19]:
pencil.senses()

[Sense('omw-en-pencil-01688604-v')]

In [20]:
pencil.senses()[0].synset()

Synset('omw-en-01688604-v')

In [21]:
pencil.synsets()  # shorthand for the above

[Synset('omw-en-01688604-v')]

In [22]:
pencil_ss = pencil.synsets()[0]
pencil_ss.definition()

'write, draw, or trace with a pencil'

In [23]:
pencil_ss.examples()

['he penciled a figure']

In [24]:
pencil_ss.hypernyms()

[Synset('omw-en-01690294-v')]

In [25]:
pencil_ss.hypernyms()[0].lemmas()

['draw']

## Taxonomy Queries

A common usage of wordnets is exploring the taxonomic structure via hypernym and hyponym relations. These operations thus have some more dedicated functions. For instance, path functions show the synsets from the starting synset to some other synset or the taxonomic root, such as [Synset.hypernym_paths()](https://wn.readthedocs.io/en/latest/api/wn.html#wn.Synset.hypernym_paths):

In [26]:
for path in pencil_ss.hypernym_paths():
    for i, ss in enumerate(path):
        print('  ' * i, ss, ss.lemmas())

 Synset('omw-en-01690294-v') ['draw']
   Synset('omw-en-01686132-v') ['represent', 'interpret']
     Synset('omw-en-01619354-v') ['re-create']
       Synset('omw-en-01617192-v') ['make', 'create']


Paths do not include the starting synset, so the length of the path (i.e., number of edges) is the length of the list of synsets. The length from a synset to the root is called the *depth*. However, as some synsets have multiple paths to the root, there is not always one single depth. Instead, the [Synset.min_depth()](https://wn.readthedocs.io/en/latest/api/wn.html#wn.Synset.min_depth) and [Synset.max_depth()](https://wn.readthedocs.io/en/latest/api/wn.html#wn.Synset.max_depth) methods find the lengths of the shortest and longest paths.

In [27]:
dog = en.synsets('dog', pos='n')[0]
len(dog.hypernym_paths())  # two paths

2

In [28]:
dog.min_depth(), dog.max_depth()

(8, 13)

It is also possible to find paths between two synsets by their lowest common hypernym (also called *least common subsumer*). Here I compare the verbs *pencil* and *pen*:

In [29]:
pen_ss = en.synsets('pen', pos='v')[0]
for path in pen_ss.hypernym_paths():
    for i, ss in enumerate(path):
        print('  ' * i, ss, ss.lemmas())

 Synset('omw-en-01697816-v') ['create verbally']
   Synset('omw-en-01617192-v') ['make', 'create']


In [30]:
pencil_ss.lowest_common_hypernyms(pen_ss)

[Synset('omw-en-01617192-v')]

In [31]:
for ss in pencil_ss.shortest_path(pen_ss):
    print(ss, ss.lemmas())

Synset('omw-en-01690294-v') ['draw']
Synset('omw-en-01686132-v') ['represent', 'interpret']
Synset('omw-en-01619354-v') ['re-create']
Synset('omw-en-01617192-v') ['make', 'create']
Synset('omw-en-01697816-v') ['create verbally']
Synset('omw-en-01698271-v') ['write', 'compose', 'pen', 'indite']


## Interlingual Queries

In Wn, each wordnet (lexicon) added to the database is given its own, independent structure. All queries that traverse across wordnets make use of the Interlingual index (ILI) on synsets.

In [32]:
pencil_ss = en.synsets('pencil', pos='n')[0]  # for this we'll use the nominal sense
pencil_ss.definition()

'a thin cylindrical pointed writing implement; a rod of marking substance encased in wood'

To get the corresponding words, senses, or synsets in some other lexicon, use the [Word.translate()](https://wn.readthedocs.io/en/latest/api/wn.html#wn.Word.translate), [Sense.translate()](https://wn.readthedocs.io/en/latest/api/wn.html#wn.Sense.translate), and [Synset.translate()](https://wn.readthedocs.io/en/latest/api/wn.html#wn.Synset.translate) functions. Of these, the function on the sense is the most natural, as it translates a specific meaning of a specific word, although all translations go through the synsets. As a word may have many senses, translating a word returns a mapping of each sense to its list of translations.

In [33]:
pencil_ss.translate(lang='it')[0].lemmas()

['lapis', 'matita']

In [34]:
pencil_ss.translate(lexicon='omw-ja')[0].lemmas()

['ペンシル', '木筆', '鉛筆']

In [35]:
en.words('pencil', pos='n')[0].translate(lexicon='omw-ja')

{Sense('omw-en-pencil-03908204-n'): [Word('omw-ja-ペンシル-n'),
  Word('omw-ja-木筆-n'),
  Word('omw-ja-鉛筆-n')],
 Sense('omw-en-pencil-14796748-n'): [Word('omw-ja-鉛筆-n')],
 Sense('omw-en-pencil-13863020-n'): [],
 Sense('omw-en-pencil-03908456-n'): [Word('omw-ja-ペンシル-n')]}

Interlingual synsets are also used to traversing relations from another wordnet. For instance, many of the lexicons in the [Open Multilingual Wordnet](https://github.com/omwn/omw-data) were created using the *expand* method where only words were translated on top of Princeton WordNet synsets. All relations (hypernyms, hyponyms, etc.) then depend on those from WordNet. In Wn, a [Wordnet](https://wn.readthedocs.io/en/latest/api/wn.html#the-wordnet-class) object may be instantiated with an `expand` parameter which selects lexicons containing such relations. By default, all lexicons are used (i.e., `expand='*'`), but you can tell Wn to not use any expand lexicons (`expand=''`) or to use a specific lexicon (`expand='omw-en:1.4'`). By being specific, you can better control the behaviour of your program, e.g., for experimental reproducibility.

In [36]:
wn.Wordnet(lexicon='omw-ja').synsets('鉛筆')[0].hypernyms()  # by default, any other installed lexicon may be used

[Synset('omw-ja-14796575-n')]

In [37]:
wn.Wordnet(lexicon='omw-ja', expand='').synsets('鉛筆')[0].hypernyms()  # disable interlingual query expansion

[]

In [38]:
wn.Wordnet(lexicon='omw-ja', expand='omw-en:1.4').synsets('鉛筆')[0].hypernyms()  # specify the expand set

[Synset('omw-ja-14796575-n')]